In [1]:
# 进入工作目录
%cd ../

/home/heyilong/codes/chinese_medical_ner_submit/project/code


In [2]:
# 读取预处理后的文本数据
train_path = "../user_data/data/train"
test_path = "../data/chusai"

from data_process.preprocess import PreProcess
from glob import glob

text_list = glob(train_path + "/*.txt") + glob(test_path + "*.txt")

texts = []
for txt_name in text_list:
    preprocess = PreProcess(txt_name)
    conll, info = preprocess.brat2conll()
    for sen in conll:
        text = "".join([c[0] for c in sen]).replace("[unused1]", " ")
        texts.append(text)



invalid literal for int() with base 10: '--ip=127.0.0.1'
K_FOLD: 10, MAX_LEN: 512
[]


In [3]:
# 读取tencent词向量中的所有word, 生成vocab
from tqdm import tqdm
vocab = []

i = 10
is_first = True
pbar = None
with open("../data/tencent_embedding/Tencent_AILab_ChineseEmbedding.txt", "r", encoding="utf8") as r:
    for line in r:
        ls = line.rstrip().split(" ", 1)
        if is_first:
            pbar = tqdm(total=int(ls[0]))
            is_first = False
            continue
        else:
            vocab.append(ls[0])
            pbar.update(1)

pbar.close()
vocab = sorted(vocab) # 按ascii排序，防止词id改变

with open("../data/tencent_embedding/vocab.txt", "w", encoding="utf8") as w:
    w.write("\n".join(vocab))

100%|██████████| 8824330/8824330 [01:44<00:00, 84551.01it/s]


In [4]:
from tqdm import tqdm
# from ruler.flashtext import KeywordProcessor
# simlpe_tencent_vocab = set()
# tree_trie = KeywordProcessor()
# tree_trie.add_keywords_from_list(vocab)

from ahocorasick import Automaton




all_text = ""
for text in tqdm(texts):
    all_text+=text

ac_tree = Automaton()
for i, word in tqdm(list(enumerate(vocab))):
    ac_tree.add_word(word, [i, word])
ac_tree.make_automaton()

simlpe_tencent_vocab = {}
for end_pos, (i, word) in ac_tree.iter(all_text):
    simlpe_tencent_vocab[word] = simlpe_tencent_vocab.get(word, 0) + 1


simlpe_tencent_vocab["</s>"]=10000
simlpe_tencent_vocab["unk"]=10000


100%|██████████| 8824330/8824330 [00:43<00:00, 203533.17it/s]


In [5]:
# 22661
len(simlpe_tencent_vocab)

22661

In [6]:
# 生成tencent简化版词向量
simlpe_tencent_embedding = []
is_first = True
pbar = None
with open("../data/tencent_embedding/Tencent_AILab_ChineseEmbedding.txt", "r", encoding="utf8") as r:
    for line in r:
        ls = line.rstrip().split(" ", 1)
        if is_first:
            pbar = tqdm(total=int(ls[0]))
            is_first = False
            continue
        else:
            if ls[0] in simlpe_tencent_vocab:
                if ls[0] == "unk":
                    ls[0] = "[UNK]"
                elif ls[0] == "</s>":
                    ls [0] = "[unused1]"
                simlpe_tencent_embedding.append(ls)
            pbar.update(1)
pbar.close()

simlpe_tencent_vocab = sorted(simlpe_tencent_vocab.items(), key=lambda x: (x[1], x[0]), reverse=True)

100%|██████████| 8824330/8824330 [01:44<00:00, 84428.67it/s]


In [9]:
# 储存相关数据
import numpy as np
tmp = []
for s in simlpe_tencent_vocab:
    if s[0] == "unk":
        s = list(s)
        s[0] = "[UNK]"
        s = tuple(s)
    elif s[0] == "</s>":
        s = list(s)
        s[0] = "[unused1]"
        s = tuple(s)
    tmp.append(s)
simlpe_tencent_vocab = tmp

simlpe_tencent_embedding_dict = {emb[0]:np.array([float(e) for e in emb[1].split(" ")]) for emb in simlpe_tencent_embedding}
simlpe_tencent_embedding_to_save = np.array([simlpe_tencent_embedding_dict[word[0]] for word in simlpe_tencent_vocab])
np.save("../user_data/data/dicts/simple_tencent_embedding.npy", simlpe_tencent_embedding_to_save)

simlpe_tencent_vocab_to_save = [f"{word} {freq}" for word, freq in simlpe_tencent_vocab]
with open("../user_data/data/dicts/simple_tencent_vocab.txt", "w", encoding="utf8") as w:
    w.write("\n".join(simlpe_tencent_vocab_to_save))


TypeError: 'tuple' object does not support item assignment

In [ ]:
"[UNK]" in simlpe_tencent_vocab

In [ ]:
simlpe_tencent_embedding_dict[]

In [ ]:
# 切词后，tencent_embedding中词的占比

import jieba
jieba.initialize("../user_data/data/dicts/simple_tencent_vocab.txt")

In [ ]:
with open()

In [ ]:
all_words = set()
tencent_words = set()
simlpe_tencent_vocab = [s[0] for s in simlpe_tencent_vocab]
for text in texts:
    for word in jieba.cut(text):
        all_words.add(word)
        if word in simlpe_tencent_vocab:
            tencent_words.add(word)

In [ ]:
len(all_words)

In [ ]:
text

In [ ]:
json.dump?

In [ ]:
import json

In [ ]:
simlpe_tencent_vocab.append("[unused1]")

In [ ]:
import json 

with open("../user_data/data/dicts/lexicon.json", "r") as r:
    words = json.load(r)

In [ ]:
import re

words = [w for w in words if (re.search("[\u4e00-\u9fa5]", w) and len(w)>1)]

In [ ]:
import json 

with open("../user_data/data/dicts/lexicon.json", "w") as w:
    json.dump(words, w, ensure_ascii=False, indent=2)

In [ ]:
words

In [ ]:
".e" in vocab